In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random

In [ ]:
###################
# Helper functions (Do not change!)

def find_index_from_time(t_obs,time,start_index=0):  
    # loop through t_obs array from i=0  
    # stopping when t_obs[i+1] is greater than time  
    # so that t_obs[i] < time < t_obs[i+1]  
    # return i 
    i=start_index
    while i+1<len(t_obs):  
        if t_obs[i+1]>time:
            break
        i=i+1
    # i now stores index corresponding to system at time requested  
    return i 
      
def resample_observations(t_obs_in, s_obs_in, t_obs_out):
    s_obs_out=[] 
    pos=0 
    for time in t_obs_out:  
        i=find_index_from_time(t_obs_in,time, start_index=pos)
        si = s_obs_in[i]  
        s_obs_out.append(si) 
        pos = i
    return s_obs_out


def gen_next_event_time(rate):
    t=random.expovariate(rate)
    return t


def random_choice_from_pdf(pdf):
    cdf=[]
    cumulative_p=0
    for p in pdf:
        cumulative_p+=p
        cdf.append(cumulative_p)
    rand=random.random()

    for i in range(len(cdf)):
        if rand<cdf[i]:
            return i
    # last cdf should be 1.0 so the following should never happen!
    print("Error generating choice, check PDF")
    return None


###################

In [ ]:
# TEMPLATE TO RUN GILLESPIE SIMULATION OF MODEL

def gillespie_toggle(s0,t_obs_out,param):

    #--0--# Unpack parameters and species variables

    = param
    
    = s0

    #--0--#

    # create arrays for output
    s_obs=[]
    t_obs=[]

    # read in start time and end time
    t_init=t_obs_out[0]
    t_final=t_obs_out[-1]

    t=t_init
    t_obs.append(t)
    s_obs.append(s0)

    while t < t_final:

        #--1--# Write labels for each event type here.

        types = [     ]

        #--1--#


        #--2--# Write rate expressions for each of the events

        
        

        #--2--#



        #--3--# Store the rates into a list preserving the order of step 1.

        rates = [     ]

        #--3--#


        #-- Do not edit the section below --#

        ## CARRY OUT GILLESPIE ALGORITHM TO STEP FORWARD TO NEXT EVENT
        ## AND UPDATE SYSTEM STATE ACCORDING TO EVENT TYPE

        # calc total reaction rate
        rate_all_events=sum(rates)

        # if rate of events is zero break from loop
        # e.g. when all reactants used up
        if rate_all_events==0:
            break

        # generate the time until the next event
        # in accordance with rate_all_events
        next_event=gen_next_event_time(rate_all_events)

        # calc PDF for event type
        # in accordance with relative rates
        pdf=[]
        for event_rate in rates:
            p_event = event_rate/sum(rates)
            pdf.append(p_event)

        rand_i =  random_choice_from_pdf(pdf)
        event_type=types[rand_i]

        # increment time and number of molecules
        # according to event type
        t=t+next_event

        #-----------------------------------#



        ## ALGORITHM HAS INCREMENTED TIME AND SELECTED NEXT EVENT
        ## WE NOW NEED TO UPDATE OUR SYSTEM ACCORDING TO THE EVENT
        ## TYPE STORED IN VARIABLE event_type


        #--4--# Complete the if-elif-else commands to update the system
              # according to event type

        if event_type==:
            
        elif event_type==:
            
        else:
            print("error unknown event type!!")

        #--4--#

        # store observation
        s=( )

        t_obs.append(t)
        s_obs.append(s)

        # loops until time t exceeds t_final

    # loop has ended

    # before we return the results we must
    # resample the output to provide observations in accordance
    # with the t_obs passed to the function
    s_obs_out=resample_observations(t_obs,s_obs,t_obs_out)
    return np.array(s_obs_out)

In [ ]:
# EXAMPLE CODE TO SIMULATE A SINGLE RUN

# define parameter values
km0L = 
kmL = 
KL = 
nL = 
kdmL = 
kpL = 
kdpL = 

km0T = 
kmT = 
KT = 
nT = 
kdmT = 
kpT = 
kdpT = 

params = [km0L, kmL, KL, nL, kdmL, kpL, kdpL,
          km0T, kmT, KT, nT, kdmT, kpT, kdpT]

# define initial conditions
ML0 = 
PL0 = 
MT0 = 
PT0 = 

s0 = [ML0, PL0, MT0, PT0]

# define time observation points
t_start = 0
t_end = 1000

t_obs = np.arange(t_start, t_end+0.1, 1)

# run simulation
s_obs = gillespie_toggle(s0, t_obs, params)

# extract the observations
ML_obs = s_obs[:,0]
PL_obs = s_obs[:,1]
MT_obs = s_obs[:,2]
PT_obs = s_obs[:,3]